In [47]:
import pandas as pd
import numpy as np

import random

import plotly.offline as py
import plotly.graph_objects as go

import networkx as nx
from networkx.readwrite import json_graph
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections

from tqdm import tqdm_notebook as tqdm

import pickle

## Some Initial Data Processing
Here's just some initial data processing. This merges two of the .csv files together and then thins out the data so that we're left with something we can visualise. The data isn't in the GitHub repository - it's too large - but you can find it here: https://www.kaggle.com/c/instacart-market-basket-analysis/data

In [53]:
train_file ='instacart_data/order_products__train.csv'
products_file = 'instacart_data/products.csv'

def return_dfs(train_file, products_file, train_percent, products_cutoff = 0, orders_q1 = 5,
              orders_q2 = 9):
    ''' Function that returns two dataframes for 2 segments of users based basket size
    Args:   train_file - the training csv file
            products_file - the products csv file
            train_percent - percentage of the train file sampled for this (smaller % makes viz possible)
            products_cutoff - only products appearing MORE often than this are included
            orders_q1 - first cutoff point for number of items in a basket
            orders_q2 - second cutoff point for number of items in a basket
    
    '''
    orders = pd.read_csv(train_file, usecols = ['order_id', 'product_id'])
    products = pd.read_csv(products_file)
    
    # Get a wide range of orders
    order_ids = orders.order_id.unique()
    # Select a sample of the orders
    order_ids = random.sample(set(order_ids), int(len(order_ids)*train_percent))
    # Reduce the size of the initial orders data
    orders = orders[orders['order_id'].isin(order_ids)]
    
    
    # Take a look at the distribution of product counts
    counts = orders.groupby('product_id').count()
    counts.rename(columns = {'order_id':'count'}, inplace = True)
    counts.reset_index(inplace = True)
    # Remove the products occuring less often that products_cutoff
    product_ids = counts.product_id[counts['count'] > products_cutoff]
    
    # Filter for baskets of a certain size
    counts = orders.groupby('order_id').count()
    counts.rename(columns = {'product_id':'count'}, inplace = True)
    counts.reset_index(inplace = True)
    # Only keep baskets below orders_q1 size and between orders_q1 and orders_q2 size
    order_ids_Q1 = counts.order_id[counts['count'] <= orders_q1]
    order_ids_Q2  = counts.order_id[(counts['count'] <= orders_q2) & (counts['count'] > orders_q1)]
    
    # Create two dataframes for the orders
    orders_small = orders[orders['order_id'].isin(order_ids_Q1)]
    orders_small = orders_small[orders_small['product_id'].isin(product_ids)]
    orders_small = orders_small.merge(products.loc[:, ['product_id', 'product_name']], how = 'left')
    # To simplify what the orders look like, I've replaced 'bag of organic bananas' with just 'bananas'
    orders_small['product_name'] = orders_small['product_name'].replace({'Bag of Organic Bananas': 'Banana'})
    orders_small['product_name'] = orders_small['product_name'].str.replace('Organic ', '')

    orders_large = orders[orders['order_id'].isin(order_ids_Q2)]
    orders_large = orders_large[orders_large['product_id'].isin(product_ids)]
    orders_large = orders_large.merge(products.loc[:, ['product_id', 'product_name']], how = 'left')

    orders_large['product_name'] = orders_large['product_name'].replace({'Bag of Organic Bananas': 'Banana'})
    orders_large['product_name'] = orders_large['product_name'].str.replace('Organic ', '')
    
    return orders_small, orders_large, order_ids_Q1, order_ids_Q2

In [54]:
# Run the function to get some ready-to-go data
orders_small, orders_large, order_ids_Q1, order_ids_Q2 = return_dfs(train_file, products_file, train_percent=0.1)

# Processing the Data for NetworkX
Here we need to create tuples comprising the paired items in the data so that we can build the graph. This code creates two sets of data, one for the "small" baskets and one for the "large" (although they're still quite small) baskets

In [59]:
paired_products_small = []

# Create the pairwise product combinations
for order_id in tqdm(order_ids_Q1):
    tmp_df = orders_small[orders_small['order_id'] == order_id]
    paired_products_small.extend(list(itertools.combinations(tmp_df.iloc[:, 2], 2)))
    
paired_products_large = []

# Create the pairwise product combinations
for order_id in tqdm(order_ids_Q2):
    tmp_df = orders_large[orders_large['order_id'] == order_id]
    paired_products_large.extend(list(itertools.combinations(tmp_df.iloc[:, 2], 2)))
    
counts_small = collections.Counter(paired_products_small)

counts_large = collections.Counter(paired_products_large)

food_df_small = pd.DataFrame(counts_small.most_common(1000),
                      columns = ['products', 'counts'])


food_df_large = pd.DataFrame(counts_large.most_common(4000),
                      columns = ['products', 'counts'])

In [56]:
# Turn one of the dataframes into a dictionary for processing into a graph
d = food_df_small.set_index('products').T.to_dict('records')
# d = food_df_large.set_index('products').T.to_dict('records')


In [58]:
# Create and populate the graph object
G = nx.Graph()

for key, val in d[0].items():
    G.add_edge(key[0], key[1], weight = val)

# Take a look at how many nodes there are in the graph; too many and it's uncomfortable to visualise
nodes = list(G.nodes)
len(nodes)

In [255]:
# Prune the plot so we only have items that are matched with at least two others
for node in nodes:
    try:
        if G.degree[node] <= 1:
            G.remove_node(node)
    except:
        print(f'error with node {node}')

nodes = list(G.nodes)
len(nodes)

312

# Build the Word2Vec model
This section of the code focuses on building the Word2Vec-like embeddings for the nodes in the network using the Deep Walk procedure. This code borrows very heavily from: https://towardsdatascience.com/deepwalk-its-behavior-and-how-to-implement-it-b5aac0290a15

In [65]:
from gensim.models import Word2Vec

In [60]:
# Read the pickle in 
with open('large_graph.pickle', 'rb') as f:
    G_large = pickle.load(f)

In [61]:
# Build a dictionary containing the weights of the edges; doing it this way saves a LOT of time in doing the probabilistic
# random walks in the next steps
for node in tqdm(G_large.nodes()):
    w_ = []
    for nodes in list(G_large.edges(str(node))):
        w_.append(G_large.get_edge_data(nodes[0], nodes[1])['weight'])
    weights[node]=w_


In [62]:
def random_walk(graph, node, weighted=False, n_steps = 5):
    ''' Function that takes a random walk along a graph'''
    local_path = [str(node),]
    target_node = node
    
    # Take n_steps random walk away from the node (can return to the node)
    for _ in range(n_steps):
        neighbours = list(nx.all_neighbors(graph, target_node))
        # See the difference between doing this with and without edge weight - it takes many, many times longer
        if weighted:
            # sample in a weighted manner
            target_node = random.choices(neighbours, weights[target_node])[0]
        else:
            target_node = random.choice(neighbours)
        local_path.append(str(target_node))
        
    return local_path
    

Now we do the random walk and then we create the node embeddings

In [63]:
walk_paths_weighted = []

i = 0
for node in tqdm(G_large.nodes()):
    for _ in range(10):
        walk_paths_weighted.append(random_walk(G_large, node, weighted=True))

In [69]:
# Instantiate the embedder
embedder_weighted = Word2Vec(window = 4, sg=1, negative=10, alpha=0.03, min_alpha=0.0001, seed=42)
# Build the vocab
embedder_weighted.build_vocab(walk_paths_weighted, progress_per=2)
# Train teh embedder to build the word embeddings- this takes a little bit of time
embedder_weighted.train(walk_paths_weighted, total_examples=embedder_weighted.corpus_count, epochs=20, report_delay=1)

(5651119, 7233600)

In [70]:
x_ = embedder_weighted.wv.most_similar("Italian Sausage Pizza", topn=10)
[i[0] for i in x_]

['Stevia',
 'Coffee Crunch In Dark Chocolate',
 'Italian Blend Shredded Cheese',
 'Bratwurst Sausage',
 "Jacob's Wonderbar Dark Roast",
 'Black Cherry Gelato',
 'Wild Chanterelle Mushroom Ravioli',
 'Cocoa Krispies Cereal',
 'Original Coffee Creamer',
 'Gunpowder Green Tea']

In [72]:
embedder_weighted.wv.most_similar('Sauvignon Blanc', topn = 10)

[('California Red Wine', 0.9043132662773132),
 ('Pinot Noir Wine', 0.8726944327354431),
 ('Pinot Grigio', 0.8328980207443237),
 ('Prosecco Sparkling Wine', 0.8056997060775757),
 ('Chardonnay Wine', 0.8010696768760681),
 ('Pinot Noir', 0.7988591194152832),
 ('Chardonnay', 0.7854331135749817),
 ("Mixed 12 Pack Lion's Share Ale", 0.7751221656799316),
 ('Brown Ale', 0.7434001564979553),
 ('India Pale Ale', 0.7353839874267578)]

In [38]:
## Save and/or load the embedding objects

# with open('embedder_weighted.pickle', 'wb') as f:
#     pickle.dump(embedder_weighted, f)

with open('embedder_weighted.pickle', 'rb') as f:
    embedder_weighted = pickle.load(f)

# Using Plotly to make the graph interactive
The code below makes an interactive map using plotly

In [73]:
# Get the nodes for plotting the graph below
nodes = [node for node in G.nodes()]
pos = nx.spring_layout(G)
len(nodes)

481

In [84]:
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []
for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    hoverinfo='text',
    text = "",
    textfont=dict(
        family="sans serif",
        size=10
    ),
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=10,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=1))

node_adjacencies = []
node_text = []
for node, adjacencies in enumerate(G.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    node_text.append(f'{nodes[node]}: {str(len(adjacencies[1]))} connections')

node_trace.marker.color = node_adjacencies
node_trace.hovertext = node_text



In [85]:
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>Graph of shopping cart items',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="some text",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )


fig.show()